In [1]:
using LorentzianSimplexSolver;

    CondaPkg Found dependencies: /Users/xiaoquer/.julia/packages/PythonCall/83z4q/CondaPkg.toml
    CondaPkg Found dependencies: /Users/xiaoquer/.julia/packages/CondaPkg/0UqYV/CondaPkg.toml
    CondaPkg Found dependencies: /Users/xiaoquer/.julia/environments/v1.11/CondaPkg.toml
    CondaPkg Initialising pixi
             │ /Users/xiaoquer/.julia/artifacts/cc02fad3b04cfb688ba4e3d60648ad8c3afe3a2d/bin/pixi
             │ init
             │ --format pixi
             └ /Users/xiaoquer/Library/Mobile Documents/com~apple~CloudDocs/Documents/Work/effective-spinfoam/code/LorentzianSimplexSolver/.CondaPkg
✔ Created /Users/xiaoquer/Library/Mobile Documents/com~apple~CloudDocs/Documents/Work/effective-spinfoam/code/LorentzianSimplexSolver/.CondaPkg/pixi.toml
    CondaPkg Wrote /Users/xiaoquer/Library/Mobile Documents/com~apple~CloudDocs/Documents/Work/effective-spinfoam/code/LorentzianSimplexSolver/.CondaPkg/pixi.toml
             │ [dependencies]
             │ numpy = "*"
             │ sympy

In [2]:
# ------------------------------------------------------------
# 1. Precision choice (user-controlled)
# ------------------------------------------------------------
const ScalarT = Float64
# const ScalarT = BigFloat

if ScalarT === BigFloat
    LorentzianSimplexSolver.PrecisionUtils.set_big_precision!(prec)
    LorentzianSimplexSolver.PrecisionUtils.set_tolerance!(sqrt(eps(BigFloat)))
else
    LorentzianSimplexSolver.PrecisionUtils.set_tolerance!(1e-10)
end

# ------------------------------------------------------------
# 2. Read simplices
# ------------------------------------------------------------
simplices = [[1,2,3,4,5], [1,2,3,5,6], [1,3,4,5,6]]

ns = length(simplices)

all_vertices = unique(Iterators.flatten(simplices))
sort!(all_vertices)

Nverts = length(all_vertices)

# ------------------------------------------------------------
# 3. Read vertex coordinates
# ------------------------------------------------------------
vertex_coords = Dict{Int, Vector{ScalarT}}()    

coords_lines = [
    "0, 0, 0, 0",
    "0, -2.7745276335252114, -0.9809436521275706, -1.6990442448471226",
    "0, 0, 0, -3.398088489694245",
    "-0.24028114141347542, -0.6936319083813028, -0.9809436521275706, -1.6990442448471226",
    "0, 0, -2.942830956382712, -1.6990442448471226",
    "0.8981365593438019, 2.7437225604241213, -0.9809436521275707, -1.6990442448471226",
]

for (v, line) in zip(all_vertices, coords_lines)
    vertex_coords[v] = LorentzianSimplexSolver.PrecisionUtils.parse_numeric_line(line, ScalarT)
end

In [3]:
# ------------------------------------------------------------
# 4. Build geometry
# ------------------------------------------------------------
datasets = LorentzianSimplexSolver.GeometryTypes.GeometryDataset{ScalarT}[]

for (s, simplex) in enumerate(simplices)
    println("\n--- Processing simplex $s with vertices $simplex ---")
    bdypoints = [vertex_coords[v] for v in simplex]
    ds = LorentzianSimplexSolver.GeometryPipeline.run_geometry_pipeline(bdypoints)
    push!(datasets, ds)
end

geom_base = LorentzianSimplexSolver.GeometryTypes.GeometryCollection(datasets)


--- Processing simplex 1 with vertices [1, 2, 3, 4, 5] ---

--- Processing simplex 2 with vertices [1, 2, 3, 5, 6] ---

--- Processing simplex 3 with vertices [1, 3, 4, 5, 6] ---


LorentzianSimplexSolver.GeometryTypes.GeometryCollection{Float64}(LorentzianSimplexSolver.GeometryTypes.GeometryDataset{Float64}[LorentzianSimplexSolver.GeometryTypes.GeometryDataset{Float64}(Matrix{ComplexF64}[[1.0163669568069527 + 0.0im -0.06055465014626238 - 0.17127441500320434im; -0.06055465014626238 + 0.17127441500320434im 1.0163669568069527 + 0.0im], [1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im], [1.1646949512180462 + 0.0im -0.06055465014626238 + 0.08563720750160216im; -0.06055465014626238 - 0.08563720750160216im 0.8680389623958591 + 0.0im], [1.0163669568069527 + 0.0im 0.1816639504387868 + 0.0im; 0.1816639504387868 + 0.0im 1.0163669568069527 + 0.0im], [0.8680389623958593 + 0.0im -0.060554650146262275 + 0.08563720750160202im; -0.060554650146262275 - 0.08563720750160202im 1.164694951218046 + 0.0im]], [[1.0660035817780522 -0.12309149097933278 0.3481553119113958 0.0; -0.12309149097933278 1.0073337313086725 -0.020742924559049587 0.0; 0.3481553119113958 -0.020742924559049587 1.058

In [4]:
# ------------------------------------------------------------
# 5. Consistency checks for each simplex
# ------------------------------------------------------------
for (idx, simplex) in enumerate(geom_base.simplex)
    LorentzianSimplexSolver.GeometryConsistency.check_sl2c_parallel_transport(simplex.solgsl2c, simplex.bdybivec55)
    LorentzianSimplexSolver.GeometryConsistency.check_so13_parallel_transport(simplex.solgso13, simplex.bdybivec4d55)
    LorentzianSimplexSolver.GeometryConsistency.check_closure_bivectors(simplex.kappa, simplex.areas, simplex.bdybivec55)
end

✓ SL(2,C) parallel transport satisfied (max residual = 3.602883265548576e-16).
✓ SO(1,3) parallel transport satisfied (max residual = 3.4713797628881078e-15)
✓ Closure condition satisfied for bivectors (max residual = 7.889616961715915e-16)
✓ SL(2,C) parallel transport satisfied (max residual = 3.4248886981654436e-16).
✓ SO(1,3) parallel transport satisfied (max residual = 4.875159404568607e-15)
✓ Closure condition satisfied for bivectors (max residual = 7.691850745534255e-16)
✓ SL(2,C) parallel transport satisfied (max residual = 2.7833528704240246e-16).
✓ SO(1,3) parallel transport satisfied (max residual = 2.5432053425292997e-15)
✓ Closure condition satisfied for bivectors (max residual = 7.787920881446914e-16)


In [5]:
# ------------------------------------------------------------
# 6. Connect simplices + face matching + gauge fixing
# ------------------------------------------------------------
if ns > 1
    println("\nFixing global κ-sign orientation ...")
    LorentzianSimplexSolver.KappaOrientation.fix_kappa_signs!(simplices, geom_base)

    println("\nBuilding global connectivity ...")
    conn = LorentzianSimplexSolver.FourSimplexConnectivity.build_global_connectivity(simplices, geom_base)
    push!(geom_base.connectivity, conn)

    geom_ref    = deepcopy(geom_base)
    geom_parity = deepcopy(geom_base)
    LorentzianSimplexSolver.FaceXiMatching.run_face_xi_matching(geom_ref; sector=:ref)
    LorentzianSimplexSolver.FaceXiMatching.run_face_xi_matching(geom_parity; sector=:parity)
    println("Global connectivity constructed for both relevant orientations.")

    # println("\nRunning final face-matching checks ...")
    LorentzianSimplexSolver.FaceMatchingChecks.check_all(geom_ref)
    LorentzianSimplexSolver.FaceMatchingChecks.check_all(geom_parity)

    println("\nPerform SU(2) and SU(1,1) gauge fixing ...")
    LorentzianSimplexSolver.GaugeFixingSU.run_su2_su11_gauge_fix(geom_ref)
    LorentzianSimplexSolver.GaugeFixingSU.run_su2_su11_gauge_fix(geom_parity)
    println("\nGauge fixing finished.")
end


Fixing global κ-sign orientation ...
[1, 2, 3]

Building global connectivity ...
  SL(2,C) matrices updated:      ✓
  SL(2,C) parity matrices updated: ✓
  Boundary bivectors updated:    ✓
  boundary ξ variables updated:  ✓
  SU(2)/SU(1,1) elements updated: ✓
  SO(1,3) frames corrected:       ✓

  SL(2,C) matrices updated:      ✓
  SL(2,C) parity matrices updated: ✓
  Boundary bivectors updated:    ✓
  boundary ξ variables updated:  ✓
  SU(2)/SU(1,1) elements updated: ✓
  SO(1,3) frames corrected:       ✓

Global connectivity constructed for both relevant orientations.

   FACE–XI GEOMETRY CHECKS

✓ Face matching satisfied (max residual = 2.5656189439360413e-16).
✓ SL(2,C) parallel transport satisfied (max residual = 3.9377191827963033e-16).
✓ Closure condition satisfied (max residual = 8.171172954064234e-16).

------------------------------
✓ All geometric consistency checks PASSED.
------------------------------


   FACE–XI GEOMETRY CHECKS

✓ Face matching satisfied (max residual = 

In [6]:
# ------------------------------------------------------------
# 7a. Symbols and action (reference orientation)
# ------------------------------------------------------------
LorentzianSimplexSolver.DefineSymbols.run_define_variables(geom_ref)

sd_ref, _ = LorentzianSimplexSolver.SolveVars.run_solver(geom_ref)

S_ref = LorentzianSimplexSolver.DefineAction.compute_action(geom_ref)

S_ref_fn, labels_ref =
    LorentzianSimplexSolver.SymbolicToJulia.build_action_function(S_ref, sd_ref)

using Symbolics
@variables γ
args_ref =
    LorentzianSimplexSolver.SymbolicToJulia.build_argument_vector(sd_ref, labels_ref, γ)

S_ref_sym = expand(simplify(S_ref_fn(args_ref...)))

(im*(-12.37647735101029 - 0.21940226591814405γ)) / γ

In [7]:
# ------------------------------------------------------------
# 7b. Symbols and action (parity orientation)
# ------------------------------------------------------------
LorentzianSimplexSolver.DefineSymbols.run_define_variables(geom_parity)

sd_parity, _ = LorentzianSimplexSolver.SolveVars.run_solver(geom_parity)

S_parity = LorentzianSimplexSolver.DefineAction.compute_action(geom_parity)

S_parity_fn, labels_parity =
    LorentzianSimplexSolver.SymbolicToJulia.build_action_function(S_parity, sd_parity)

args_parity =
    LorentzianSimplexSolver.SymbolicToJulia.build_argument_vector(sd_parity, labels_parity, γ)

S_parity_sym = expand(simplify(S_parity_fn(args_parity...)))

(im*(-12.376477351010294 + 0.21940226591814638γ)) / γ

In [46]:
phase = expand(simplify((S_ref_sym+S_parity_sym)//2))

(-12.376477351010292im) / γ

In [54]:
label_pos = "S^(" * repeat("+", ns) * ")"
label_neg = "S^(" * repeat("-", ns) * ")"

"S^(---)"

In [53]:
imag(simplify(S_ref_sym - phase)),  imag(simplify(S_parity_sym - phase))

(-0.21940226591814405, 0.21940226591814638)

In [44]:
# ------------------------------------------------------------
# One triangle (i,j)
# ------------------------------------------------------------
function regge_face(tetareasign_ij, area_ij, Na, Nb)
    θ = LorentzianSimplexSolver.Dihedral.theta_ab(Na, Nb)  # Float64
    area = tetareasign_ij == -1 ? -area_ij : area_ij
    theta_out = tetareasign_ij == -1 ? pi - θ : θ
    return area * theta_out
end

# ------------------------------------------------------------
# One 4-simplex
# ------------------------------------------------------------
function regge_simplex(area_mat, tetareasign_mat, N_mat)
    S = 0
    for i in 1:4
        for j in (i+1):5
            S += regge_face(
                tetareasign_mat[i][j],
                area_mat[i][j],
                N_mat[i],
                N_mat[j]
            )
        end
    end
    return S
end

# ------------------------------------------------------------
# All simplices
# ------------------------------------------------------------
function run_Regge_action_1(geom)
    ns = length(geom.simplex)

    area_all = [geom.simplex[k].areas for k in 1:ns]

    tetareasign_all = [geom.simplex[k].tetareasign for k in 1:ns]
    N_all = [geom.simplex[k].tetnormalvec for k in 1:ns]

    S = 0
    for k in 1:ns
        S += regge_simplex(area_all[k], tetareasign_all[k], N_all[k])
    end
    return S
end

run_Regge_action_1 (generic function with 2 methods)

In [51]:
result = run_Regge_action_1(geom_ref)

0.2194022659179462

In [7]:
# ------------------------------------------------------------
# 8. EOMs
# ------------------------------------------------------------
dS = LorentzianSimplexSolver.EOMsHessian.compute_EOMs(S, sd)

grad_fns = LorentzianSimplexSolver.SymbolicToJulia.build_gradient_functions(dS, sd)

LorentzianSimplexSolver.EOMsHessian.check_EOMs(grad_fns, sd; γ = one(ScalarT));

LoadError: InterruptException:

In [ ]:
# ------------------------------------------------------------
# 9. Hessian computations
# ------------------------------------------------------------
H = LorentzianSimplexSolver.EOMsHessian.compute_Hessian(S, sd)

hess_fns = LorentzianSimplexSolver.SymbolicToJulia.build_hessian_functions(H, sd)
H_eval, _ = LorentzianSimplexSolver.EOMsHessian.evaluate_hessian(hess_fns, sd; γ = one(ScalarT))

H_det = det(H_eval)